# model

In [1]:
import torch
import torch.nn as nn
class DiRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, dropout_rate):
        super(DiRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        
        # bidirectional设为True即得到双向循环神经网络
        self.encoder = nn.LSTM(input_size = embed_size, 
                    hidden_size = num_hiddens, 
                    num_layers = num_layers,
                    dropout = dropout_rate,
                    bidirectional = False)
        self.decoder = nn.Linear(num_hiddens, vocab_size) # 初始时间步和最终时间步的隐藏状态作为全连接层输入

    def forward(self, inputs):
        # inputs的形状是(批量大小, 词数)，因为LSTM需要将序列长度(seq_len)作为第一维，所以将输入转置后
        # 再提取词特征，输出形状为(词数, 批量大小, 词向量维度)
        embeddings = self.embedding(inputs.permute(1, 0))
        # rnn.LSTM只传入输入embeddings，因此只返回最后一层的隐藏层在各时间步的隐藏状态。
        # outputs形状是(词数, 批量大小, 2 * 隐藏单元个数)
        outputs, _ = self.encoder(embeddings) # output, (h, c)
        # 连结初始时间步和最终时间步的隐藏状态作为全连接层输入。它的形状为
        # (批量大小, 2 * 隐藏单元个数)。
#         print(outputs.shape)
        output = outputs.permute(1, 0, 2)
#         print(output.shape)
        outs = output.reshape(output.size(0)*output.size(1), output.size(2))
#         print(outs.shape)
        ret = self.decoder(outs)
        return ret

# data

In [2]:
import torch
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

In [3]:
class Corpus(object):
    def __init__(self, train_batch_size=20, eval_batch_size=10, bptt=35):
        self.bptt = bptt
        train_iter = WikiText2(split='train')
        self.tokenizer = get_tokenizer('basic_english')
        counter = Counter()
        for line in train_iter:
            counter.update(self.tokenizer(line))
        self.vocab = Vocab(counter)
        train_iter, val_iter, test_iter = WikiText2()
        train_data = self.data_process(train_iter)
        val_data = self.data_process(val_iter)
        test_data = self.data_process(test_iter)

        self.train_data = self.batchify(train_data, train_batch_size)
        self.val_data = self.batchify(val_data, eval_batch_size)
        self.test_data = self.batchify(test_data, eval_batch_size)

    def data_process(self, raw_text_iter):
        data = [torch.tensor([self.vocab[token] for token in self.tokenizer(item)],
                           dtype=torch.long) for item in raw_text_iter]
        return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

    def batchify(self, data, batch_size):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Divide the dataset into batch_size parts.
        nbatch = data.size(0) // batch_size
        # Trim off any extra elements that wouldn't cleanly fit (remainders).
        data = data.narrow(0, 0, nbatch * batch_size)
        # Evenly divide the data across the batch_size batches.
        data = data.view(batch_size, -1).t().contiguous()
        return data.to(device)

    def get_batch(self, source, i):
        seq_len = min(self.bptt, len(source) - 1 - i)
        data = source[i:i+seq_len]
        target = source[i+1:i+1+seq_len].reshape(-1)
        return data, target

    def get_ntokens(self):
        return len(self.vocab.stoi)

# main

In [4]:
import argparse
import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable

import data
import model
class args():
    emsize = 100
    nhid = 200
    nlayers = 5
    lr = 2
    epochs = 2000
    batch_size = 32
    bptt = 256
    dropout = 0.5
    tied = False
    seed = 1234
    cuda = True
    log_interval = 100
    clip = 0.01
    save = 'model.pt'

In [5]:
data_loader = Corpus(train_batch_size=args.batch_size,
                     eval_batch_size=args.batch_size,
                     bptt=args.bptt)

In [6]:
train_data = data_loader.train_data
val_data = data_loader.val_data
test_data = data_loader.test_data

In [7]:
def get_batch(source, i, evaluation=False):
    seq_len = min(args.bptt, len(source) - 1 - i)
#     data = Variable(source[i:i+seq_len], volatile=evaluation)
#     target = Variable(source[i+1:i+1+seq_len].view(-1))
    data = source[i:i+seq_len].t()
    tmp = source[i+1:i+1+seq_len].t()
#     print(data.shape,tmp.shape)
    target = tmp.reshape(-1)
    return data, target
#     return data.clone().detach(), target.clone().detach()

In [8]:
# import torch
# torch.__version__
# print(train_data.size(0))
# for batch, i in enumerate(range(0, train_data.size(0) - 1, args.bptt)):
#     data, targets = get_batch(train_data, i)
#     print(data.shape,targets.shape,data[-1],targets[-3])
#     print(data.requires_grad)
#     if (batch>=2): break

In [9]:
len(data_loader.vocab.itos)

28783

In [10]:
ntokens = len(data_loader.vocab.itos)
# model = RNNModel(args.model, ntokens, args.emsize, args.nhid,
#         args.nlayers, args.dropout, args.tied)
model = DiRNN(ntokens, args.emsize, args.nhid, args.nlayers, args.dropout)

# Glove

In [11]:
import os
import torchtext.vocab as Vocab
glove_vocab = Vocab.GloVe(name='6B', dim=100, cache=os.path.join("data","glove"))

In [12]:
len(glove_vocab.itos), glove_vocab.stoi['999th']

(400000, 265062)

In [13]:
for i, word in enumerate(data_loader.vocab.itos):
    print(i, word)
    if i>10: break

0 <unk>
1 <pad>
2 the
3 ,
4 .
5 of
6 and
7 in
8 to
9 a
10 =
11 was


In [14]:
def load_pretrained_embedding(l, pretrained_vocab):
    """从预训练好的vocab中提取出words对应的词向量"""
    embed = torch.zeros(l, pretrained_vocab[0].shape[0]) # 初始化为0
    oov_count = 0 # out of vocabulary
    for i, word in enumerate(data_loader.vocab.itos):
        try:
            idx = pretrained_vocab.stoi[word]
            embed[i, :] = pretrained_vocab.vectors[idx]
        except KeyError:
            oov_count += 1
    if oov_count > 0:
        print("There are %d oov words." % oov_count)
#     for i in range(len(vec)):
#         embed[i] = vec[i]
    return embed

model.embedding.weight.data.copy_(load_pretrained_embedding(ntokens, glove_vocab))
model.embedding.weight.requires_grad = True # 直接加载预训练好的, 所以不需要更新它

There are 1028 oov words.


In [15]:
if args.cuda:
    model = model.to(torch.device('cuda'))

In [16]:
model.eval()
model.train()

DiRNN(
  (embedding): Embedding(28783, 100)
  (encoder): LSTM(100, 200, num_layers=5, dropout=0.5)
  (decoder): Linear(in_features=200, out_features=28783, bias=True)
)

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)

In [18]:
def clip_gradient(optimizer, grad_clip):
    for group in optimizer.param_groups:
        for param in group["params"]:
            if param.grad is not None:
                param.grad.data.clamp_(-grad_clip, grad_clip)

def evaluate(data_source):
    model.eval()
    total_loss = 0
    total_words = 0
    for i in range(0, data_source.size(0) - 1, args.bptt):
        data, targets = get_batch(data_source, i)
        
        output = model(data)
        loss = criterion(output, targets)

        total_loss += loss.data * targets.shape[0]
        total_words += targets.shape[0]
    total_loss = float(total_loss.to(torch.device('cpu')))
#     print(total_loss,total_words)
    return total_loss / total_words, math.exp(total_loss / total_words)

def train():
    model.train()
    total_loss = 0.
    total_words = 0
    for batch, i in enumerate(range(0, train_data.size(0) - 1, args.bptt)):
        data, targets = get_batch(train_data, i)
        model.zero_grad()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, targets)
        loss.backward()
        clip_gradient(optimizer,args.clip)
        optimizer.step()

        total_loss += loss.data * targets.shape[0]
        total_words += targets.shape[0]
        
#         if (batch % args.log_interval == 0 and batch > 0):
#             cur_loss = total_loss / args.log_interval
#             elapsed = time.time() - start_time
#             print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
#                     'loss {:5.2f} | ppl {:8.2f}'.format(
#                 233, batch, len(train_data) // args.bptt, args.lr,
#                 elapsed * 1000 / args.log_interval, cur_loss, math.exp(cur_loss)))
#             total_loss = 0
#             start_time = time.time()

    cur_loss = total_loss / total_words
    print('train {:10d} words | loss {:5.5f} | ppl {:5.5}'.format(total_words, cur_loss, math.exp(cur_loss)))
    return cur_loss, math.exp(cur_loss)

In [19]:
import numpy as np
Losstrain = []
Lossval = []
Losstest = []

In [20]:
import time
for T in range(args.epochs):
    print('Round : ',T,"  ",time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))
#     train()
#     print('  valid ',evaluate(val_data))
#     print('  test  ',evaluate(test_data))
    trainl, trainp = train()
    vall, valp = evaluate(val_data)
    testl, testp = evaluate(test_data)
    
    Losstrain.append(trainp)
    Lossval.append(valp)
    Losstest.append(testp)
    np.save('model-glove-clip-train',np.array(Losstrain))
    np.save('model-glove-clip-val',np.array(Lossval))
    np.save('model-glove-clip-test',np.array(Losstest))
    
    model.embedding.weight.data.copy_(load_pretrained_embedding(ntokens, glove_vocab))
    
    print('  valid {:5.5f} | test {:5.5f}'.format(valp, testp))
    print('-'*80)

Round :  0    2021-05-20 00:31:47
train    2049952 words | loss 7.38463 | ppl 1611.0
There are 1028 oov words.
  valid 963.73146 | test 915.99806
--------------------------------------------------------------------------------
Round :  1    2021-05-20 00:32:53
train    2049952 words | loss 7.03183 | ppl 1132.1
There are 1028 oov words.
  valid 897.05280 | test 843.17546
--------------------------------------------------------------------------------
Round :  2    2021-05-20 00:34:00
train    2049952 words | loss 6.99516 | ppl 1091.3
There are 1028 oov words.
  valid 889.02721 | test 848.87013
--------------------------------------------------------------------------------
Round :  3    2021-05-20 00:35:06
train    2049952 words | loss 6.97678 | ppl 1071.5
There are 1028 oov words.
  valid 861.12471 | test 811.92497
--------------------------------------------------------------------------------
Round :  4    2021-05-20 00:36:13
train    2049952 words | loss 6.96857 | ppl 1062.7
There a

train    2049952 words | loss 5.87813 | ppl 357.14
There are 1028 oov words.
  valid 291.76821 | test 270.33791
--------------------------------------------------------------------------------
Round :  39    2021-05-20 01:15:12
train    2049952 words | loss 5.85608 | ppl 349.35
There are 1028 oov words.
  valid 280.90359 | test 260.56222
--------------------------------------------------------------------------------
Round :  40    2021-05-20 01:16:19
train    2049952 words | loss 5.83372 | ppl 341.63
There are 1028 oov words.
  valid 279.60180 | test 258.76393
--------------------------------------------------------------------------------
Round :  41    2021-05-20 01:17:26
train    2049952 words | loss 5.81224 | ppl 334.37
There are 1028 oov words.
  valid 273.15095 | test 253.22686
--------------------------------------------------------------------------------
Round :  42    2021-05-20 01:18:33
train    2049952 words | loss 5.79053 | ppl 327.19
There are 1028 oov words.
  valid 271

train    2049952 words | loss 5.36377 | ppl 213.53
There are 1028 oov words.
  valid 197.45455 | test 183.33875
--------------------------------------------------------------------------------
Round :  75    2021-05-20 01:55:26
train    2049952 words | loss 5.35546 | ppl 211.76
There are 1028 oov words.
  valid 195.85513 | test 182.02985
--------------------------------------------------------------------------------
Round :  76    2021-05-20 01:56:33
train    2049952 words | loss 5.34683 | ppl 209.94
There are 1028 oov words.
  valid 195.92629 | test 182.29718
--------------------------------------------------------------------------------
Round :  77    2021-05-20 01:57:40
train    2049952 words | loss 5.33738 | ppl 207.97
There are 1028 oov words.
  valid 195.26262 | test 182.00812
--------------------------------------------------------------------------------
Round :  78    2021-05-20 01:58:47
train    2049952 words | loss 5.32832 | ppl 206.09
There are 1028 oov words.
  valid 192

train    2049952 words | loss 5.10413 | ppl 164.7
There are 1028 oov words.
  valid 171.03599 | test 160.01922
--------------------------------------------------------------------------------
Round :  111    2021-05-20 02:35:40
train    2049952 words | loss 5.09818 | ppl 163.72
There are 1028 oov words.
  valid 168.13208 | test 157.14227
--------------------------------------------------------------------------------
Round :  112    2021-05-20 02:36:47
train    2049952 words | loss 5.09170 | ppl 162.67
There are 1028 oov words.
  valid 167.99321 | test 157.33596
--------------------------------------------------------------------------------
Round :  113    2021-05-20 02:37:54
train    2049952 words | loss 5.08702 | ppl 161.91
There are 1028 oov words.
  valid 168.79092 | test 157.87209
--------------------------------------------------------------------------------
Round :  114    2021-05-20 02:39:01
train    2049952 words | loss 5.08038 | ppl 160.83
There are 1028 oov words.
  valid 

train    2049952 words | loss 5.15894 | ppl 173.98
There are 1028 oov words.
  valid 168.67245 | test 157.88873
--------------------------------------------------------------------------------
Round :  147    2021-05-20 03:15:55
train    2049952 words | loss 5.05095 | ppl 156.17
There are 1028 oov words.
  valid 164.53608 | test 154.66379
--------------------------------------------------------------------------------
Round :  148    2021-05-20 03:17:02
train    2049952 words | loss 5.01502 | ppl 150.66
There are 1028 oov words.
  valid 160.66708 | test 150.99240
--------------------------------------------------------------------------------
Round :  149    2021-05-20 03:18:09
train    2049952 words | loss 4.99366 | ppl 147.48
There are 1028 oov words.
  valid 158.50800 | test 148.93305
--------------------------------------------------------------------------------
Round :  150    2021-05-20 03:19:16
train    2049952 words | loss 4.97789 | ppl 145.17
There are 1028 oov words.
  valid

...

train    2049952 words | loss 4.15911 | ppl 64.014
There are 1028 oov words.
  valid 146.99551 | test 138.81764
--------------------------------------------------------------------------------
Round :  1049    2021-05-21 05:43:59
train    2049952 words | loss 4.15757 | ppl 63.916
There are 1028 oov words.
  valid 146.63116 | test 138.48087
--------------------------------------------------------------------------------
Round :  1050    2021-05-21 05:46:08
train    2049952 words | loss 4.15687 | ppl 63.871
There are 1028 oov words.
  valid 150.55452 | test 141.32366
--------------------------------------------------------------------------------
Round :  1051    2021-05-21 05:48:17
train    2049952 words | loss 4.15665 | ppl 63.857
There are 1028 oov words.
  valid 150.58481 | test 141.21194
--------------------------------------------------------------------------------
Round :  1052    2021-05-21 05:50:27
train    2049952 words | loss 4.15584 | ppl 63.806
There are 1028 oov words.
  v

train    2049952 words | loss 4.13407 | ppl 62.431
There are 1028 oov words.
  valid 148.52978 | test 139.46292
--------------------------------------------------------------------------------
Round :  1085    2021-05-21 07:01:24
train    2049952 words | loss 4.13379 | ppl 62.414
There are 1028 oov words.
  valid 148.80128 | test 139.84682
--------------------------------------------------------------------------------
Round :  1086    2021-05-21 07:03:32
train    2049952 words | loss 4.13361 | ppl 62.402
There are 1028 oov words.
  valid 146.82632 | test 138.58928
--------------------------------------------------------------------------------
Round :  1087    2021-05-21 07:05:41
train    2049952 words | loss 4.13408 | ppl 62.432
There are 1028 oov words.
  valid 148.16733 | test 139.57939
--------------------------------------------------------------------------------
Round :  1088    2021-05-21 07:07:51
train    2049952 words | loss 4.13167 | ppl 62.282
There are 1028 oov words.
  v

train    2049952 words | loss 4.11676 | ppl 61.36
There are 1028 oov words.
  valid 149.69244 | test 141.23391
--------------------------------------------------------------------------------
Round :  1121    2021-05-21 08:18:51
train    2049952 words | loss 4.11571 | ppl 61.296
There are 1028 oov words.
  valid 149.22749 | test 141.05611
--------------------------------------------------------------------------------
Round :  1122    2021-05-21 08:21:00
train    2049952 words | loss 4.11438 | ppl 61.214
There are 1028 oov words.
  valid 148.97440 | test 139.94068
--------------------------------------------------------------------------------
Round :  1123    2021-05-21 08:23:08
train    2049952 words | loss 4.11451 | ppl 61.222
There are 1028 oov words.
  valid 149.21026 | test 140.67442
--------------------------------------------------------------------------------
Round :  1124    2021-05-21 08:25:18
train    2049952 words | loss 4.11376 | ppl 61.176
There are 1028 oov words.
  va

train    2049952 words | loss 4.12799 | ppl 62.053
There are 1028 oov words.
  valid 149.80805 | test 141.24012
--------------------------------------------------------------------------------
Round :  1157    2021-05-21 09:36:18
train    2049952 words | loss 4.12261 | ppl 61.72
There are 1028 oov words.
  valid 151.44875 | test 142.45917
--------------------------------------------------------------------------------
Round :  1158    2021-05-21 09:38:27
train    2049952 words | loss 4.27827 | ppl 72.116
There are 1028 oov words.
  valid 360.87984 | test 337.02249
--------------------------------------------------------------------------------
Round :  1159    2021-05-21 09:40:36
train    2049952 words | loss 4.42399 | ppl 83.428
There are 1028 oov words.
  valid 159.12300 | test 148.76970
--------------------------------------------------------------------------------
Round :  1160    2021-05-21 09:42:46
train    2049952 words | loss 4.22617 | ppl 68.455
There are 1028 oov words.
  va

KeyboardInterrupt: 